In [7]:
#User-based Recommendation Example using Cosine Similarity

In [8]:
import pandas as pd
import numpy as np

In [9]:
users = pd.read_csv('ml-100k/u.user', sep='|', names=['user_id','age','gender','occupation','zip_code'])
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id','item_id','rating','timestamp'])
movies = pd.read_csv('ml-100k/u.item', sep='|', names=['movie_id','movie_title','release_date','video_release_date','IMDb URL','unknown','Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'], encoding='latin-1')
movies = movies.iloc[:,[0,1]]

In [10]:
#merge
merge1 = pd.merge(movies,ratings,right_on='item_id',left_on='movie_id')
finalDF = pd.merge(merge1,users,left_on='user_id',right_on='user_id')
finalDF.head()

,movie_id,movie_title,user_id,item_id,rating,timestamp,age,gender,occupation,zip_code
0,1,Toy Story (1995),308,1,4,887736532,60,M,retired,95076
1,4,Get Shorty (1995),308,4,5,887737890,60,M,retired,95076
2,5,Copycat (1995),308,5,4,887739608,60,M,retired,95076
3,7,Twelve Monkeys (1995),308,7,4,887738847,60,M,retired,95076
4,8,Babe (1995),308,8,5,887736696,60,M,retired,95076


In [11]:
movies.head()

,movie_id,movie_title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [12]:
# Association Matrix - Pivot Table
# pivotTable

learningMatrix = ratings.pivot_table(index=['item_id'],columns=['user_id'],values='rating').reset_index(drop=True)
learningMatrix.fillna(0,inplace=True)
learningMatrix.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
0,5.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,4.0,...,2.0,3.0,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
1,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#apply Cosine Similarity Formula on Association Matrix
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import hamming
from scipy.spatial.distance import minkowski


from sklearn.metrics import pairwise_distances

#Pairwise distance only accept Matrix object


movie_similarity = 1 - pairwise_distances( learningMatrix, metric="cosine" )
np.fill_diagonal( movie_similarity, 0 ) 

ratings_matrix = pd.DataFrame( movie_similarity )
ratings_matrix.head()

#movie_similarity object your model

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.000000,0.402382,0.330245,0.454938,0.286714,0.116344,0.620979,0.481114,0.496288,0.273935,...,0.035387,0.0,0.000000,0.000000,0.035387,0.0,0.0,0.0,0.047183,0.047183
1,0.402382,0.000000,0.273069,0.502571,0.318836,0.083563,0.383403,0.337002,0.255252,0.171082,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.078299,0.078299
2,0.330245,0.273069,0.000000,0.324866,0.212957,0.106722,0.372921,0.200794,0.273669,0.158104,...,0.000000,0.0,0.000000,0.000000,0.032292,0.0,0.0,0.0,0.000000,0.096875
3,0.454938,0.502571,0.324866,0.000000,0.334239,0.090308,0.489283,0.490236,0.419044,0.252561,...,0.000000,0.0,0.094022,0.094022,0.037609,0.0,0.0,0.0,0.056413,0.075218
4,0.286714,0.318836,0.212957,0.334239,0.000000,0.037299,0.334769,0.259161,0.272448,0.055453,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.094211


In [18]:
try:
    user_inp=input('Enter the reference movie title based on which recommendations are to be made: ')
    inp=movies[movies['movie_title']==user_inp].index.tolist()
    inp=inp[0]

    movies['similarity'] = ratings_matrix.iloc[inp]
    #movies.head(5)
    print("Recommended movies based on your choice of ",user_inp ,": \n", movies.sort_values( ["similarity"], ascending = False )[1:10])
    
except:
    print(movies.head(10))

Enter the reference movie title based on which recommendations are to be made: Toy Story (1995)
Recommended movies based on your choice of  Toy Story (1995) : 
      movie_id                                   movie_title  similarity
180       181                     Return of the Jedi (1983)    0.699925
120       121                 Independence Day (ID4) (1996)    0.689786
116       117                              Rock, The (1996)    0.664555
404       405                    Mission: Impossible (1996)    0.641322
150       151  Willy Wonka and the Chocolate Factory (1971)    0.638158
221       222               Star Trek: First Contact (1996)    0.636727
99        100                                  Fargo (1996)    0.630601
236       237                          Jerry Maguire (1996)    0.624075
173       174                Raiders of the Lost Ark (1981)    0.622382
